# 실행 전 런타임 유형을 GPU로 변경 (Runtime > Change Runtime time) >> GPU

In [0]:
# 구글 드라이브와 Colab 연동
from google.colab import drive
drive.mount('/content/drive') # 'Go to this URL in a browser URL'에 접속하여 이용동의 후 인증코드 복사해서 붙여넣기

In [0]:
# 작업 경로를 구글 드라이브로 변경
import os
os.chdir('/content/drive/My Drive/PyConKorea2019-Tutorials/LRP')
os.getcwd()

In [0]:
# 사용할 모듈을 설치(tensorflow, keras, numpy, matplotlib등 자주 사용하는 모듈은 이미 코랩에 설치되어 있음 )
!pip install innvestigate

In [0]:
# 사용할 전체 모듈을 불러옴
import innvestigate
import innvestigate.utils
import numpy as np
import matplotlib.pyplot as plt
import PIL

from keras.applications import vgg16, imagenet_utils

In [0]:
# 시각화할 이미지 선택
image_path = "husky.jpg"

In [0]:
# 이미지를 불러옴
img = PIL.Image.open(image_path)
img = img.resize((224, 224))
plt.imshow(img)
plt.show()

In [0]:
# 2차원 이미지를 CNN학습이 가능하도록 batch, channel이 추가된 4차원 배열로 전처리(preprocessing)
img_array = np.array(img)
img_array = img_array[np.newaxis, :]
img_array = img_array.astype(np.float)
img_array = imagenet_utils.preprocess_input(img_array)

In [0]:
# 전치리된 형태확인
img_array.shape

In [0]:
# 전처리 된 이미지를 시각화   
plt.imshow(img_array[0]) # imshow는 3차원까지만 시각화 가능하여 (224, 224, 3)차원을 시각화
plt.show() # RGB 3channel을 사용하여 표현

In [0]:
# 각 채널을 따로 출력해서 보면 아래와 같음
plt.subplot(1,3,1)
plt.imshow(img_array[0,:,:,0],cmap="jet")
plt.subplot(1,3,2)
plt.imshow(img_array[0,:,:,1],cmap="jet")
plt.subplot(1,3,3)
plt.imshow(img_array[0,:,:,2],cmap="jet")
plt.show() 

In [0]:
# VGG모델로 학습된 모델을 불러와 softmax를 통과하지 않은(without) 값을 불러옴
model = vgg16.VGG16()
model = innvestigate.utils.model_wo_softmax(model)

In [0]:
# 'deep_taylor' 방식을 활용한 분석기(analyer)를 생성하고 적용함
analyzer = innvestigate.create_analyzer("deep_taylor", model)
a = analyzer.analyze(img_array)

*  'deep_taylor' 대신 'deconvnet', 'lrp', 'lrp.z', 'lrp.z_IB', 'lrp.epsilon',  'lrp.alpha_1_beta_0', 'lrp.alpha_1_beta_0_IB', 'lrp.z_plus', 'lrp.z_plus_fast', 'lrp.sequential_preset_a', 'lrp.sequential_preset_b', 'lrp.sequential_preset_a_flat', 'lrp.sequential_preset_b_flat', 'deep_taylor', 'deep_taylor.bounded', 'deep_lift.wrapper', 'pattern.net', 'pattern.attribution' 사용가능

*  각 모델의 자세한 사용법은 다음 링크활용
https://github.com/albermax/innvestigate/blob/master/examples/notebooks/imagenet_compare_methods.ipynb



In [0]:
# 3개인 color 채널을 다시 한개로 통합함
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))

# 값을 [-1, 1] 사이로 정규화함
a /= np.max(np.abs(a))

# 결과물을 출력
plt.imshow(a[0], cmap="seismic", clim=(-1, 1)) # heatmap형식으로 출력하도록 'seismic'설정